In [1]:
import fitz  # PyMuPDF
import pyttsx3  # Text-to-speech library
import speech_recognition as sr  # Speech recognition library
import tkinter as tk
from tkinter import scrolledtext

In [2]:
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [3]:
def extract_text_from_pdf(pdf_path, start_page=0):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(start_page, len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [4]:
def find_answer(text, keyword):
    keyword_index = text.lower().find(keyword.lower())
    if keyword_index != -1:
        answer_start_index = keyword_index + len(keyword)
        # Find the end of the sentence (ending with a period)
        answer_end_index = text.find(".", answer_start_index)
        if answer_end_index == -1:
            answer_end_index = len(text)
        answer = text[answer_start_index:answer_end_index].strip()
        return answer
    else:
        return "Answer not found in the PDF"

In [5]:
def listen_for_input(prompt):
    recognizer = sr.Recognizer()
    speak(prompt)
    with sr.Microphone() as source:
        print(f"Listening for {prompt.lower()}...")
        audio = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio)
        print(f"{prompt}: {text}\n")
        return text
    except sr.UnknownValueError:
        print("Sorry, I didn't catch that. Please try again.\n")
        return None
    except sr.RequestError:
        print("Sorry, I encountered an issue with my speech recognition service. Please try again later.\n")
        return None


In [ ]:
def handle_question():
    user_question = listen_for_input("Question")
    if user_question and 'end' in user_question.lower():
        print("Stopping...")
        root.destroy()
        return
    
    user_keyword = listen_for_input("What is the keyword associated with this question?")
    if user_keyword:
        extracted_answer = find_answer(pdf_text, user_keyword)
        speak(f"Extracted Answer: {extracted_answer}")
        result_display.insert(tk.END, f"Question: {user_question}\n")
        result_display.insert(tk.END, f"Keyword: {user_keyword}\n")
        result_display.insert(tk.END, f"Keyword: {extracted_answer}\n\n")

In [ ]:
def main():
    global pdf_text, result_display, root
    pdf_path = 'introduction-to-natural-language-processing.pdf'  # Path to your PDF file
    start_page = 10  # Page numbers are zero-based; 10 means starting from the 11th page
    
    # Extract text from PDF once at the beginning
    pdf_text = extract_text_from_pdf(pdf_path, start_page)
    
    # Initialize Tkinter root window
    root = tk.Tk()
    root.title("PDF Q&A System")

    frame = tk.Frame(root)
    frame.pack(padx=10, pady=10)

    button = tk.Button(frame, text="Ask Question", command=handle_question)
    button.pack(pady=5)

    result_display = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=80, height=20)
    result_display.pack(padx=5, pady=5)

    root.mainloop()

if __name__ == "__main__":
    main()